In [4]:
import os
import pygmt
import numpy as np
import xesmf as xe
import pandas as pd
import xarray as xr
from scipy import interpolate

from xarray import DataArray
from scipy.ndimage import gaussian_filter
from numpy import deg2rad, sin, cos, meshgrid, gradient

%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
def regridding1(carbdata):
    ds_in = xr.DataArray(
        dims=["latitude", "longitude"],
        coords={"latitude": np.arange(-90,90.05,0.05), "longitude": np.arange(-180,180.05,0.05)},
    )
    ds_out = xe.util.grid_2d(-180.0, 180.0, 0.1, -90.0, 90.0, 0.1)
    regrid = xe.Regridder(ds_in, ds_out, 'nearest_s2d', periodic=True, weights='data/nearest_s2d_gridder_res1.nc')
    return regrid(carbdata)

def regridding2(ds):
    ds_out = xe.util.grid_2d(-180.0, 180.0, 0.25, -90.0, 90.0, 0.25)
    regrid = xe.Regridder(ds, ds_out, 'nearest_s2d', periodic=True, weights='data/nearest_s2d_gridder_res2.nc')
    return regrid(ds)

def getData(timeclim, CO2curve='smooth'):
    abiotic = xr.open_dataset('env_var/enviVar'+str(timeclim)+'Ma_res1_'+CO2curve+'.nc')
    continent = abiotic.flood.fillna(10)
    continent = continent.where(continent>4)
    abiotic['elevation'] = abiotic.bathy.copy()
    shelf = xr.open_dataset('data/shelfs/'+str(timeclim)+'Ma.nc')
    shelf2 = regridding1(shelf)
    shelf.close()
    abiotic['shelf'] = (('latitude', 'longitude'), shelf2.flood.values)
    ensemble = xr.open_dataset('photozoan.res2/proj_'+str(timeclim)+'Ma/proj_'+str(timeclim)+'Ma_photozoan.res2_ensemble.tif')
    return abiotic, ensemble, continent

def earth_radius(lat):
    a = 6378137
    b = 6356752.3142
    e2 = 1 - (b**2/a**2)
    lat = deg2rad(lat)
    lat_gc = np.arctan( (1-e2)*np.tan(lat) )
    r = (
        (a * (1 - e2)**0.5) 
         / (1 - (e2 * np.cos(lat_gc)**2))**0.5 
        )
    return r

def area_grid(lat, lon):
    xlon, ylat = meshgrid(lon, lat)
    R = earth_radius(ylat)
    dlat = deg2rad(gradient(ylat, axis=0))
    dlon = deg2rad(gradient(xlon, axis=1))
    dy = dlat * R
    dx = dlon * R * cos(deg2rad(ylat))
    area = dy * dx
    xda = DataArray(
        area,
        dims=["latitude", "longitude"],
        coords={"latitude": lat, "longitude": lon},
        attrs={
            "long_name": "area_per_pixel",
            "description": "area per pixel",
            "units": "m^2",
        },
    )
    return xda

In [7]:
curve = 'foster'
percfile = 'data/percentPhotozoan_foster.csv'
df = pd.read_csv(percfile)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../environment/data/percentPhotozoan_foster.csv'

In [ ]:
times = df['Time (Ma)'].values

In [ ]:
if os.path.isfile('suitability_Foster.nc'):
    #tmp = np.load('suitAreaFoster.npz')
    #lats = tmp['lat']
    #times = tmp['time']
    #val = tmp['data']
    ncdata = xr.open_dataset('suitability_Foster.nc')
    
else:    
    suitArea = np.zeros((len(times),180))
    for t in range(len(times)):
        timeclim = int(times[t])

        abiotic, ensemble, continent = getData(timeclim, CO2curve=curve)
        flooddata = regridding2(abiotic)

        nda = xr.DataArray(
                            ensemble.isel(band=4).band_data.values,
                            dims=["latitude", "longitude"],
                            coords={"latitude": ensemble.y.values, "longitude": ensemble.x.values},
                        )

        dataEM = nda.to_dataset(name='wmean')
        tmp = nda.fillna(0).values
        dataEM['wmean'] = (('latitude', 'longitude'), gaussian_filter(tmp,1))

        dataEM['flood'] = (('latitude', 'longitude'), np.flipud(flooddata.flood.values))
        dataEMFlood = dataEM.where(dataEM.flood==1)
        da_area = area_grid(abiotic['latitude'], abiotic['longitude'])

        regrid = xe.Regridder(dataEM, abiotic, 'bilinear', periodic=True, weights='data/bilinear_gridder_reef.nc') 
        ensembleReef = regrid(dataEM)
        abiotic['area'] = da_area.copy()*1.e-6
        abiotic['wMean'] = ensembleReef.wmean.copy()/1000.

        shelfdata = abiotic.where(abiotic.flood==1) 
        reef = shelfdata.where(shelfdata.wMean>0).wMean
        reefArea = (shelfdata.area*reef).copy()

        divlat_mean2 = reefArea.sum(dim='longitude')/1.e5
        suitArea[t,:] = divlat_mean2.coarsen(latitude=10).sum().values

        reefArea.close()
        abiotic.close()
        ensemble.close()
        continent.close()
        ensembleReef.close()
        da_area.close()
        shelfdata.close()
        reef.close()
        
    newtimes = np.linspace(0, 265, 266)
    newSuitNearest = np.zeros((266,180))
    for r in range(0,180):
        interp1 = interpolate.interp1d(df['Time (Ma)'].values, suitArea[:,r], kind = "nearest")
        newSuitNearest[:,r] = interp1(newtimes)

    newSuitCubic = np.zeros((266,180))
    for r in range(0,180):
            interp2 = interpolate.interp1d(df['Time (Ma)'].values, suitArea[:,r], kind = "cubic")
            newSuitCubic[:,r] = interp2(newtimes)
    #np.savez('suitAreaFoster.npz',data=newSuit,
    #         lat=divlat_mean2.coarsen(latitude=10).mean().latitude.values,
    #         time=newtimes
    #        )
    lats = divlat_mean2.coarsen(latitude=10).mean().latitude.values
    nda = xr.DataArray(
                    np.fliplr(newSuitNearest).T,
                    dims=["latitude","times"],
                    coords={"times": -newtimes, "latitude": lats},
                    )
    ncdata = nda.to_dataset(name='linear')
    ncdata['cubic'] = (('latitude', 'times'), np.fliplr(newSuitCubic).T)
    ncdata.to_netcdf('suitability_Foster.nc')

In [ ]:
suitability = ncdata.where(ncdata.suit_foster>0.625).suit_foster

In [ ]:
# Plot figure
fig = pygmt.Figure()
pygmt.config(
    MAP_FRAME_WIDTH="0.5p",
    MAP_FRAME_TYPE="fancy",
)
font = "4p,Helvetica-Bold"

with pygmt.config(FONT='20p,Helvetica,black',COLOR_BACKGROUND="white"):
    fig.basemap(region=[-265, 0, -60, 60], projection="X40c/16c", frame='af')
    pygmt.makecpt(cmap="panoply", series=[0, 10], reverse=False, 
                  )
    fig.grdimage(
        grid=suitability, cmap=True, nan_transparent=True# M=True
    )
    fig.colorbar(position="JMR")
#fig.savefig(fname="freq_over_time.png")
fig.show(dpi=500, width=1000)